In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from R125_R32_PR3 import configuration

# Load data from csv
data = pd.read_csv('R32R125new2.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(1669700)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.2688)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.7312)
    
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'R125'].fix(3.5)
    m.fs.properties.PR_kappa_A['R125', 'R32'].fix(-3.5)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'R125'].fix(-3.5)
    m.fs.properties.PR_kappa_B['R125', 'R32'].fix(3.5)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
#     m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
#     m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].unfix()
    m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].unfix()
    
#     m.fs.kappas = Constraint(expr=m.fs.properties.PR_kappa['R32', 'R125'] == 
#                     m.fs.properties.PR_kappa['R125', 'R32'])
#     m.fs.properties.PR_kappa['R32', 'R125'] = Expression(expr=value(m.fs.properties.PR_kappa['R125', 'R32']))
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-10)
    m.fs.properties.PR_kappa_A['R32', 'R125'].setub(10)

    m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-10)
    m.fs.properties.PR_kappa_A['R125', 'R32'].setub(10)

    m.fs.properties.PR_kappa_B['R32', 'R125'].setlb(-10)
    m.fs.properties.PR_kappa_B['R32', 'R125'].setub(10)

    m.fs.properties.PR_kappa_B['R125', 'R32'].setlb(-10)
    m.fs.properties.PR_kappa_B['R125', 'R32'].setub(10)
    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 1669700, "x_R32":0.7312, 
             "x_R125":0.2688, "y_R125":0.2501, "y_R32":0.7499}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

# Check that degrees of freedom is 0
# assert degrees_of_freedom(m) == 0

2021-08-24 16:58:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 0


In [7]:
variable_name = ["fs.properties.PR_kappa_A['R32', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'R32']",
                 "fs.properties.PR_kappa_B['R32', 'R125']",
                 "fs.properties.PR_kappa_B['R125', 'R32']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-9

# def SSE(m, data):
#     expr = (
#             (float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2
# #             + (float(data["x_R125"]) -
# #              m.fs.state_block.mole_frac_phase_comp["Liq", "R125"])**2
#            )
    
#     return expr*1e4


In [9]:
pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

obj_value, parameters = pest.theta_est()

2021-08-24 16:58:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-24 16:58:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-24 16:58:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-24 16:58:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-24 16:59:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-24 16:59:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-24 16:59:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-24 16:59:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-24 16:59:08 [INFO] idaes

ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

In [ ]:
pest.ef_instance.display()

In [ ]:
print(large_residuals_set(pest.ef_instance))